In [1]:
import numpy as np
import argparse
from tqdm.notebook import tqdm
import gym
import safety_gym
import env as _env
from models import ProbEnsemble, PredictEnv
from cem.ccem import ConstrainedCEM
from mbrl_utils import *
from batch_utils import *
from sac import ReplayMemory

from models import GaussianPolicy
from utils import Args


pygame 2.0.1 (SDL 2.0.14, Python 3.6.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: cannot import name 'ProbEnsemble'

In [ ]:
env_name = 'Safexp-PointGoal1-v0'
env = gym.make(env_name)

obs_shape = env.observation_space.shape
act_shape = env.action_space.shape
state_size = np.prod(obs_shape)
action_size = np.prod(act_shape)
policy = GaussianPolicy(obs_shape[0], act_shape[0])

In [6]:
args = Args(
    replay_size = 2000000,
    epoch_length = 1000,
    hidden_size = 200,
    env = env_name,
    cuda = True,
    learn_cost = True,
)

In [7]:
cost_size = 1 if args.learn_cost else 0

env_sampler = EnvSampler(env, max_path_length=args.epoch_length)
env_pool = ReplayMemory(args.replay_size)
env_model = ProbEnsemble(state_size, action_size, network_size=5,
                         reward_size=1+cost_size, hidden_size=args.hidden_size)
cem_agent = ConstrainedCEM(env)
cem_agent.set_model(env_model)
env_model.to('cuda')
predict_env = PredictEnv(env_model, args.env)


/home/andrew/Documents/offline_safe_rl/models/ensemble.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(val, dtype=torch.float32)


In [8]:
def rollout(steps, max_eps_length, env, agent):
    obs = env.reset()
    done = False
    eps_ret = 0
    t = 0
    
    obs_buf = []
    act_buf = []
    eps_ret_buf = []
    eps_len_buf = []
    
    for step in tqdm(range(steps)):
        action = agent.select_action(obs)
        next_obs, reward, done, info = env.step(action)
        obs_buf.append(obs)
        act_buf.append(action)
        eps_ret += reward
        cost = info.get("cost", 0)
        
        obs = next_obs
        

        if done or step == steps - 1 or t == max_eps_length - 1:
            eps_ret_buf.append(eps_ret)
            eps_len_buf.append(t + 1)
            
            obs = env.reset()
            done = False
            eps_ret = 0
            t = 0
        else:
            t += 1

        if step == steps - 1:
            break
            
    
    return dict(states=obs_buf, actions=act_buf, eps_ret=eps_ret_buf, eps_len=eps_len_buf)


## Return of 2.5 is good

In [12]:
wandb.init(project='safety-gym',
           group="testing",
           config=args)

for epoch in range(100):
    exploration_before_start(args, env_sampler, env_pool, cem_agent)
    train_predict_model(args, env_pool, predict_env)
    cem_rollout = rollout(1000, args.epoch_length, env, cem_agent)
    ret = np.mean(cem_rollout["eps_ret"])
    wandb.log({"CEM/return": ret})
    print(f"Epoch {epoch}, CEM Return {ret}")
    

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chaisilva (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.25 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0, CEM Return -0.004093102357461742


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1, CEM Return 0.36938211136373034


...

Epoch 97, CEM Return 0.0023911389868475075


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 98, CEM Return -0.5256214368520691


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 99, CEM Return 0.5990601255455175
